In [1]:
import numpy as np
import threading
from copy import deepcopy
from tqdm import tqdm_notebook as tqdm
import tic_tac_toe as ttt

In [2]:
reward = {
    'win':75,
    'lose':-100,
    'next_move':-1,
    'wrong_move':-50
}

MUTATE_PROB = 0.6
ADD_LAYER_PROB = 0.01

In [3]:
def board_to_state(board):
    return np.concatenate((board.reshape(1,-1),(np.array(1)).reshape(1,1)), axis=1)

def state_to_board(state):
    return state.reshape(3,3)

def next_move(board,pos,p):
    return ttt.next_move(board,pos//3,pos%3,p)

In [4]:
board = np.zeros((3,3))

In [5]:
class cpu_player:
    def __init__(self):
        self.score = 0
        self.w = [np.random.normal(0,10,(10,9))]
        self.wins = 0
        self.losses = 0
        self.wrongs = 0
        
    def reset_scores(self):
        self.score = 0
        self.wins = 0
        self.losses = 0
        self.wrongs = 0
        
    def get_ratio_score(self):
        return self.wins/(self.losses + self.wrongs*10 + 10e-5)
        
    def compute_next_move(self,board):
        if(board.shape == (3,3)):
            return self.compute_next_move(board_to_state(board))
        
        for i in range(len(self.w)):
            board = board @ self.w[i]
            if(i<len(self.w)-1):
                board = np.tanh(board)
            else:
                return board.argmax()
    
    def mutate(self):
        for i in range(len(self.w)):
            mutate_mask = np.random.uniform(0,1,self.w[i].shape)>MUTATE_PROB
            mutation = np.random.normal(0,1,self.w[i].shape)
            mutation = np.ma.array(mutation, mask=mutate_mask).filled(0)
            self.w[i]+=mutation
        if(np.random.uniform(0,1)<ADD_LAYER_PROB):
            self.w.append(np.random.normal(0,1,(9,9)))

In [6]:
def play_game_2cpus(p1, p2):    
    board = np.zeros((3,3))
    
    p=1
    win = False
    move = 1
    while(move<=9 and not win):
#         print(board)
        if(p==1):
#             print(1)
            pos = p1.compute_next_move(board)
        else:
#             print(2)
            pos = p2.compute_next_move(board * -1)

#         print(p,pos)
        if(not next_move(board,pos,p)):
            if(p==1):
                p1.score += reward['wrong_move']
                p1.wrongs += 1
            else:
                p2.score += reward['wrong_move']
                p2.wrongs += 1
            break
        else:
            if(p==1):
                p1.score += reward['next_move']
            else:
                p2.score += reward['next_move']
            p*=-1
            win = ttt.win_condition(board)
            move+=1
    
    if(win==1):
        p1.score += reward['win']
        p1.wins += 1
        p2.losses += 1
        p2.score += reward['lose']
    elif(win==2):
        p2.score += reward['win']
        p2.wins += 1
        p1.losses += 1
        p1.score += reward['lose']
#     print(board)
#     if(win==1):
#         print("!!!!!!!!!!!!!!!! " + p1 + " wins !!!!!!!!!!!!!!!!")
#     else:
#         print("!!!!!!!!!!!!!!!! " + p2 + " wins !!!!!!!!!!!!!!!!")

In [7]:
def genetic_algorithm(n_players=100, n_gen=100, n_cycle_per_gen=20, new=True):
    if(n_players%2!=0):
        raise Exception("n_players must be even")
    global players
    global best_score
    global avg_score
    if(new):
        players = [cpu_player() for i in range(n_players)]
        best_score = []
        avg_score = []
    index_array = np.arange(n_players)

    generations = tqdm(range(n_gen))
    for gen in generations:
        for player in players:
            player.reset_scores()
#         print("Generation", gen, " Started")
        
        for cycle in range(n_cycle_per_gen):
            games = []
            np.random.shuffle(index_array)
            for game in range(n_players//2):
                games.append(threading.Thread(target=play_game_2cpus, 
                                        args = [players[index_array[2*game]],players[index_array[2*game+1]]]))
            for game in games:
                game.start()
            for game in games:
                game.join()
        
        players.sort(key=lambda x: x.score, reverse=True)
        del players[n_players//2:]
        for i in range(n_players//2):
            players.append(deepcopy(players[i]))
            players[n_players//2+i].mutate()
        best_score.append(players[0].score)
        avg_score.append(sum([player.score for player in players])/len(players))
        generations.set_description("Best player: "+ str(best_score[-1]) + " Avg Score: " + str(avg_score[-1]))

In [ ]:
genetic_algorithm(n_players=500, n_gen=1000, n_cycle_per_gen=50)

In [ ]:
# for player in players:
#     print(player.score, player.wins, player.losses, player.wrongs)

In [ ]:
from matplotlib import pyplot as plt


plt.plot(best_score)
plt.plot(avg_score)
plt.show()

In [ ]:
def play_game_vs_cpu(p1):    
    board = np.zeros((3,3))
    
    p=-1
    win = False
    move = 1
    while(move<=9 and not win):
        print(board)
        if(p==1):
            pos = p1.compute_next_move(board)
        else:
            pos = int(input("Which position? "))

        if(not next_move(board,pos,p)):
            print("WRONG MOVE")
            break
        else:
            p*=-1
            win = ttt.win_condition(board)
            move+=1

    print(board)

In [ ]:
play_game_vs_cpu(players[0])

In [ ]:
for player in players:
    print(len(player.w))

In [ ]:
(players[1].w)[0]

In [ ]:
np.concatenate((board.reshape(1,-1),(np.array(1)).reshape(1,1)), axis=1)

In [ ]:
board.reshape(1,-1)